# 数据质量验证

https://docs.greatexpectations.io/docs/home/

https://greatexpectations.io/expectations/

https://greatexpectations.io/integrations/

## 示例目标

验证一份订单数据文件 `orders.csv` 是否满足以下质量要求：

| 质量规则 | 描述 |
|--------|------|
| 1. 表非空 | 至少有 1 条记录 |
| 2. 列存在性 | 必须包含 `order_id`, `user_id`, `amount`, `status`, `created_at` |
| 3. 唯一性 | `order_id` 必须唯一 |
| 4. 非空检查 | `user_id`, `amount`, `created_at` 不能有空值 |
| 5. 数值范围 | `amount` 应在 0.01 到 10000 之间 |
| 6. 枚举值检查 | `status` 只能是 `pending`, `paid`, `shipped`, `cancelled` |
| 7. 日期格式 | `created_at` 应为合法的 ISO 日期时间格式 |

## 准备工作

1. 安装 Great Expectations

    ```bash
    pip install great-expectations
    # 或更完整安装（含 pandas、numpy 等）
    pip install "great-expectations[all]"
    ```

2. 初始化 GE 项目

    ```bash
    great_expectations init
    ```

    会创建目录结构：
    ```
    great_expectations/
    ├── expectations/
    ├── checkpoints/
    ├── datasources/
    ├── uncommitted/
    │   ├── validations/
    │   └── documentation/
    └── great_expectations.yml
    ```

## 示例数据文件：`orders.csv`

```csv
order_id,user_id,amount,status,created_at
1001,101,299.50,paid,2025-04-05T10:20:30Z
1002,102,150.00,pending,2025-04-05T11:15:00Z
1003,101,,shipped,2025-04-05T12:00:00Z
1004,103,9999.99,paid,2025-04-05T13:45:00Z
1005,,50.00,cancelled,2025-04-05T14:20:00Z
1006,104,-50.00,paid,2025-04-05T15:10:00Z
1007,105,300.00,unknown,2025-04-05T16:00:00Z
```

> ⚠️ 包含多个质量问题：空值、负数、非法状态

## 执行

### 步骤 1：添加数据源（Datasource）

```bash
great_expectations datasource new
```

选择：
- What data would you like to connect to? → **Pandas**
- Give your new Datasource a short name → `csv_datasource`

然后输入文件路径：
```
Path to the root directory where your data files are stored: ./data
```

会自动创建配置。

### 步骤 2：创建 Expectation Suite

```bash
great_expectations suite new
```

选择：
- How would you like to create your Expectation Suite? → **manual**

输入套件名：
```
Name the new Expectation Suite [orders_suite] → orders_suite
```

### 步骤 3：编写 Expectations（Python 交互式）

系统会打开一个 Jupyter Notebook（或 Python shell），你可以在其中编写期望。

### 示例代码（在 Notebook 中运行）：

```python
import great_expectations as gx
from great_expectations.core.batch import BatchRequest

context = gx.get_context()

# 构建 Batch Request
batch_request = BatchRequest(
    datasource_name="csv_datasource",
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name="orders",
    batch_spec_passthrough={"reader_method": "read_csv"}
)

# 获取 Validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="orders_suite"
)

# 添加期望
validator.expect_table_row_count_to_be_between(min_value=1, max_value=None)

validator.expect_column_to_exist("order_id")
validator.expect_column_to_exist("user_id")
validator.expect_column_to_exist("amount")
validator.expect_column_to_exist("status")
validator.expect_column_to_exist("created_at")

validator.expect_column_values_to_not_be_null("user_id")
validator.expect_column_values_to_not_be_null("amount")
validator.expect_column_values_to_not_be_null("created_at")

validator.expect_column_values_to_be_unique("order_id")

validator.expect_column_values_to_be_between(
    column="amount",
    min_value=0.01,
    max_value=10000.00
)

validator.expect_column_values_to_be_in_set(
    column="status",
    value_set=["pending", "paid", "shipped", "cancelled"]
)

validator.expect_column_values_to_match_strftime_format(
    column="created_at",
    strftime_format="%Y-%m-%dT%H:%M:%SZ"
)

# 保存期望套件
validator.save_expectation_suite(discard_failed_expectations=False)

print("✅ Expectation Suite 已保存：orders_suite")
```

### 步骤 4：创建并运行 Checkpoint

```bash
great_expectations checkpoint new my_checkpoint orders_suite
```

编辑生成的 `checkpoints/my_checkpoint.yml`，确保 batch_request 正确。

然后运行校验：

```bash
great_expectations checkpoint run my_checkpoint
```

### 输出结果

控制台

```text
Validation failed!
Success: False
Expectation Suite: orders_suite
Total Expectations: 12
Successful: 7
Unsuccessful: 5
```

## 查看 HTML 报告

打开：
```
great_expectations/uncommitted/documentation/local_site/index.html
```

点击 `orders_suite` 查看详细报告，包含：

- 哪些规则失败
- 失败的样本数据（如 `amount` 为空、为负数）
- 统计信息和图表

## 修复数据后重试

修改 `orders.csv` 中的问题后重新运行：

```bash
great_expectations checkpoint run my_checkpoint
```

预期输出：
```text
Validation succeeded! 🎉
```

## 高级技巧

自定义 Expectation（Python）

```python
from great_expectations.execution_engine import PandasExecutionEngine
from great_expectations.expectations.expectation import ColumnMapExpectation
from great_expectations.expectations.metrics import column_condition_metrics

class ExpectColumnValuesToBePositive(ColumnMapExpectation):
    """Expect column values to be > 0."""
    map_metric = "column_values.positive"
    success_keys = ()

@PandasExecutionEngine.column_condition_metric
def column_values_positive(cls, column, **kwargs):
    return column > 0

# 使用
validator.expect_column_values_to_be_positive(column="amount")
```

## 与 Airflow / DolphinScheduler 集成

将校验封装为脚本 `validate_orders.py`，在调度系统中调用：

```python
from great_expectations.data_context import DataContext

context = DataContext()
result = context.run_checkpoint(checkpoint_name="my_checkpoint")

if not result.success:
    raise ValueError("❌ 数据质量校验失败！")
else:
    print("✅ 校验通过")
```

## 总结：核心 API 一览

| 类型 | 方法示例 |
|------|---------|
| 表级 | `.expect_table_row_count_to_be_between()` |
| 列存在 | `.expect_column_to_exist()` |
| 非空 | `.expect_column_values_to_not_be_null()` |
| 唯一性 | `.expect_column_values_to_be_unique()` |
| 范围 | `.expect_column_values_to_be_between()` |
| 枚举 | `.expect_column_values_to_be_in_set()` |
| 正则/格式 | `.expect_column_values_to_match_regex()` / `..._match_strftime_format()` |
| 分布比较 | `.expect_column_kl_divergence_to_be_less_than()` |

**下一步建议**：

- 将 GE 集成到你的 ETL 流程中，作为“质量守门员”
- 使用 `ActionListValidationOperator` 自动发送告警邮件
- 结合 OpenMetadata 展示数据质量结果和血缘

需要我为你生成一个 **Spark 版本** 或 **数据库表校验示例** 吗？欢迎继续提问！

# 集成 DolphinScheduler

非常好！将 **Great Expectations（GE）校验脚本** 集成到 **DolphinScheduler** 是实现自动化数据质量管控的关键一步。以下是 **详细、可落地的集成方案**，包含脚本模板、DolphinScheduler 配置说明和最佳实践。

---

## ✅ 一、集成目标

在 DolphinScheduler 工作流中插入一个“数据质量校验”任务，实现：

- 自动运行 GE 校验
- 校验失败时任务失败，阻断后续流程
- 支持查看报告或日志
- 可扩展告警机制（邮件、钉钉等）

---

## ✅ 二、前置条件

| 项目 | 要求 |
|------|------|
| DolphinScheduler 版本 | ≥ 1.3.9 或 2.x/3.x |
| Worker 节点环境 | 安装 Python ≥ 3.7，并安装 `great_expectations` 包 |
| GE 项目路径 | 所有 Worker 节点都能访问（建议统一挂载或部署） |

```bash
# 在所有 DolphinScheduler Worker 节点执行
pip install great-expectations
```

> 📁 建议路径：`/opt/dolphinscheduler/scripts/great_expectations/`

---

## ✅ 三、GE 校验脚本模板（`validate_data.py`）

```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
GE 数据质量校验脚本（适用于 DolphinScheduler）
"""

import sys
import os
from great_expectations.data_context import DataContext
from pathlib import Path

# 设置 GE 项目根目录（根据实际情况修改）
GE_ROOT_DIR = "/opt/dolphinscheduler/scripts/great_expectations"

def run_validation():
    try:
        # 加载 GE 上下文
        context = DataContext(context_root_dir=GE_ROOT_DIR)

        # 运行 Checkpoint（替换为你自己的 checkpoint 名称）
        checkpoint_result = context.run_checkpoint(
            checkpoint_name="orders_checkpoint"
        )

        # 输出结果
        print(f"✅ 校验完成: {checkpoint_result.success}")
        print(f"📍 报告路径: {checkpoint_result.get_metrics('validation_result_url')}")

        # 判断是否成功
        if checkpoint_result.success:
            print("🎉 数据质量校验通过")
            sys.exit(0)
        else:
            print("❌ 数据质量校验失败！")
            # 可选：打印失败详情
            for result in checkpoint_result.run_results.values():
                vr = result.get("validation_result")
                if vr and not vr.success:
                    for result in vr.results:
                        if not result.success:
                            print(f"  ❌ {result.expectation_config.expectation_type}: {result.expectation_config.kwargs}")
            sys.exit(1)

    except Exception as e:
        print(f"🚨 校验过程出错: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    run_validation()
```

---

## ✅ 四、DolphinScheduler 配置步骤

### 步骤 1：上传脚本到服务器

将 `validate_data.py` 和整个 `great_expectations/` 项目目录复制到所有 Worker 节点的指定路径：

```bash
scp -r great_expectations/ user@worker1:/opt/dolphinscheduler/scripts/
scp validate_data.py user@worker1:/opt/dolphinscheduler/scripts/
```

> 💡 建议使用 NFS、Git 同步或 CI/CD 工具保持一致性

---

### 步骤 2：在 DolphinScheduler 中创建“Python 任务”

1. 登录 DolphinScheduler Web UI
2. 进入项目 → 工作流定义 → 创建工作流
3. 添加一个 **Python 任务**
4. 配置如下：

| 字段 | 值 |
|------|----|
| 任务名称 | `ge_data_quality_check` |
| Python 脚本 |  
```python
import sys
sys.path.append('/opt/dolphinscheduler/scripts')
from validate_data import run_validation
run_validation()
```  
或直接写完整路径脚本内容 |
| Python 版本 | `python3`（确保是安装 GE 的环境） |
| 任务优先级 | MEDIUM |
| 失败重试 | 建议 1 次 |
| 资源 | 可上传脚本为“资源文件”并引用 |

> ⚠️ 如果使用“资源文件”，需先上传 `validate_data.py` 到 DS 资源中心，然后在任务中引用。

---

### 步骤 3：设置任务依赖

```text
[Spark 任务] --> [GE 数据校验] --> [下游 Hive 导入]
```

- 只有 GE 校验通过，才执行后续任务
- 若 GE 失败，流程中断，触发告警

---

## ✅ 五、增强功能（可选）

### 1. 动态传参（如日期分区）

在 DolphinScheduler 中使用 **全局参数**，如 `${bizdate}`

```python
# 在 validate_data.py 中接收参数
import json

# 示例：从命令行获取参数
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--date", default="2025-04-05")
args = parser.parse_args()

# 在 batch_request 中使用
batch_request = {
    "datasource_name": "my_csv",
    "data_connector_query": {"partition_request": {"runtime_parameters": {"date": args.date}}}
}
```

然后在 DolphinScheduler 的 Python 脚本中调用：

```bash
python /opt/dolphinscheduler/scripts/validate_data.py --date ${bizdate}
```

👉 使用 **Shell 任务** 调用更灵活。

---

### 2. 发送报告链接

将生成的 HTML 报告上传到内部 Web 服务器或对象存储，然后在日志中输出 URL：

```python
print("📊 报告查看: http://intranet.reports/ge/orders/20250405.html")
```

运维人员可在 DS 任务日志中点击链接查看。

---

### 3. 失败告警

在 DolphinScheduler 中配置：
- 告警组（邮件、钉钉、企业微信、Webhook）
- 当任务失败时自动通知负责人

---

## ✅ 六、替代方案：使用 Shell 任务（更灵活）

如果你不想用 Python 任务，可以用 **Shell 任务** 直接调用：

```bash
cd /opt/dolphinscheduler/scripts/great_expectations
python -m great_expectations checkpoint run orders_checkpoint
```

> ⚠️ 注意：需确保当前 Python 环境已安装 GE

---

## ✅ 七、验证与调试

1. 在 Worker 节点手动运行脚本测试：
   ```bash
   python /opt/dolphinscheduler/scripts/validate_data.py
   ```
2. 查看 DolphinScheduler 任务日志：
   - 成功：`sys.exit(0)`
   - 失败：`sys.exit(1)` → 会被识别为失败
3. 检查报告是否生成：
   ```
   great_expectations/uncommitted/documentation/local_site/validations/...
   ```

---

## ✅ 八、最佳实践总结

| 项目 | 建议 |
|------|------|
| 部署方式 | 所有 Worker 统一安装 GE 和脚本 |
| 脚本管理 | 使用 Git + CI/CD 同步更新 |
| 报告存储 | 定期归档 HTML 报告，避免占用过多空间 |
| 日志输出 | 打印关键信息（成功/失败、报告路径） |
| 错误处理 | 捕获异常并 `sys.exit(1)`，防止误判成功 |
| 权限控制 | 确保 Worker 有读取数据源（数据库、文件）的权限 |

---

## 🎯 示例效果

```text
DolphinScheduler 工作流：
1. [Spark] 清洗订单数据
2. [Hive] 写入 DWD 层
3. [Python] 运行 GE 校验
   └─ ✅ 通过 → 继续
   └─ ❌ 失败 → 告警 + 终止
4. [Email] 通知 BI 团队数据已就绪
```

---

## 📦 附件：目录结构建议

```bash
/opt/dolphinscheduler/scripts/
├── great_expectations/
│   ├── expectations/
│   ├── checkpoints/
│   ├── datasources/
│   └── great_expectations.yml
├── validate_data.py
└── requirements.txt
```

---

🎯 **下一步建议**：

- 将此脚本模板封装为通用组件，支持多表校验
- 结合 OpenMetadata 展示血缘 + 质量结果
- 使用 `ActionListValidationOperator` 自动发送失败通知

需要我为你生成一个 **支持多 checkpoint 动态调用的通用脚本** 或 **DolphinScheduler 与 GE + OpenMetadata 联合部署方案** 吗？欢迎继续提问！

In [ ]:
import pandas as pd
from great_expectations.data_context.data_context.context_factory import get_context
import great_expectations as gx

# ----------------------------
# 1. 加载数据（示例）
# ----------------------------
data = {
    'customer_name': ['张三', '', '李四', None, '未知'],
    'id_card': ['11010119900101001X', '', 'N/A', None, 'UNKNOWN'],
    'phone': ['13800138000', '000-000-0000', '-', None, '13900139000'],
    'birth_date': ['1990-01-01', '1900-01-01', '1985-06-15', '1970-01-01', None],
    'address': ['北京市朝阳区', '暂无', '', 'N/A', '上海市浦东新区']
}
df = pd.DataFrame(data)
# 初始化 GX Context
context = gx.get_context()

# 直接获取 Validator（无需 datasource/asset）
validator = context.get_validator(
    dataframe=df,
    expectation_suite_name="field_completeness_suite",
    overwrite_existing=True,
)